# Analog quantum computation: tutorial

This tutorial shows how to execute an analog computation. One first prepares a job, that contains a Schedule (a description of the time-dependent Hamiltonian) and (optionally) an observable to be measured at the end of the simulation.

## 1. Time-independent case

Here, we take a simple example of a time-independent Hamiltonian 
$$H = \sigma_x $$
and measure $\langle \sigma_z \rangle$ after a time $t = \pi$.


<div class="alert alert-block alert-info">
- What result do you expect to see for $\langle \sigma_z(t)\rangle$?
</div>

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# optional, uncomment only for Google colab

# !pip install myqlm
# !git clone https://github.com/tayral/phy580_2026.git
# %cd phy580_2026
# !ls

In [ ]:
import matplotlib.pyplot as plt
from qat.core import Schedule, Observable
from basic_qutip_qpu import QutipQPU

# define an operator (here sigma_z acting on qubit 0, total number of qubits is 1)
H = Observable.x(0, 1)
sched = Schedule(drive = [(1, H)], tmax=3.14)
# (in general, the drive is a list of pairs of the form (lambda(t), P)
# where lambda(t) is a (possibly) time-dependent coefficient and P an operator (observable) )

# instantiate the QPU (the simulator of quantum computer)
qpu = QutipQPU()

# construct a job from the schedule
# a job is a schedule + an observable (here the observable is Z)
job = sched.to_job(observable=Observable.z(0, 1))
# submit : res contains the results
res = qpu.submit(job)

In [ ]:
# final value of the observable.
print("<Z(tf)> = ", res.value)

In [ ]:
# the simulator also computes the values of <Z> at intermediate times
times = [float(x) for x in res.value_data.keys()]
values = [res.value_data[x].re for x in res.value_data.keys()]
plt.plot(times, values, "-")
plt.xlabel("$t$")
plt.ylabel(r"$\langle Z\rangle$");

<div class="alert alert-block alert-info">
- What the numerical result match your expectation?
</div>

### Job with no observable: sampling job

If no observable is specified in the job, then one returns samples from the final state (i.e Z measurements) with the corresponding probabilities:

In [ ]:
sampling_job = sched.to_job()
# submit : res contains the results
sampling_res = qpu.submit(sampling_job)
for sample in sampling_res:
    print(sample.state, sample.probability)

## 2. Time-dependent example

Let us now look at a time-dependent Hamiltonian. Here we just vary linearly the coefficient in front of the $\sigma_x$ term:
$$H = t \sigma_x. $$

In [ ]:
from qat.core import Variable
t = Variable("t", float)  # we create a time variable

sched2 = Schedule(drive = [(2.*t, H)], tmax=3.14)

job2 = sched2.to_job(observable=Observable.z(0, 1))
res2 = qpu.submit(job2)

In [ ]:
times2 = [float(x) for x in res2.value_data.keys()]
values2 = [res2.value_data[x].re for x in res2.value_data.keys()]
plt.plot(times2, values2, "-", label=r"$\langle Z(t) \rangle$, time-dependent case")
plt.plot(times, values, "--", label=r"$\langle Z(t) \rangle$, time-independent case")

plt.xlabel("$t$")
plt.ylabel(r"$\langle Z\rangle$");
plt.legend();

## 3. (Bonus) Using a noisy QPU: Lindblad decoherence

Here, we add noise in the form of jump operators $$L = \sqrt{\gamma} \sigma_z. $$

We keep $H = \sigma_x $ from the first section.

In [ ]:
import numpy as np
from qat.hardware import HardwareModel

# define a "HardwareModel" which contains jump operators
gamma = 0.1
hw_model = HardwareModel(jump_operators=[np.sqrt(gamma) * Observable.sigma_z(0, 1)])

# define a QPU containing this hardware model
noisy_qpu = QutipQPU(hardware_model=hw_model)

# define a constant Hamiltonian
H = Observable.x(0, 1)
sched = Schedule(drive = [(1, H)], tmax=20.)
job = sched.to_job(observable=Observable.z(0, 1))

res_noisy = noisy_qpu.submit(job)

res_noiseless = qpu.submit(job)

# the simulator also computes the values of <Z> at intermediate times
for name, result in [("noiseless", res_noiseless), ("noisy", res_noisy)]: 
    print(f"<Z> ({name})= {result.value}")
    times = [float(x) for x in result.value_data.keys()]
    values = [result.value_data[x].re for x in result.value_data.keys()]
    plt.plot(times, values, "-", label=name)
plt.xlabel("$t$")
plt.ylabel(r"$\langle Z\rangle$");
plt.grid()
plt.legend();

<div class="alert alert-block alert-info">
- How does decoherence modify the time evolution of $\sigma_z(t)$ ?
</div>

### Using quantum trajectories

The default mode to solve the Lindblad equation is to store the density matrix ("deterministic" mode). The second mode (which ca be used by setting the ``sim_method`` parameter to "stochastic") generates quantum trajectories instead. Its number is controlled by the ``ntraj`` parameter.

In [ ]:
gamma = 0.1
hw_model = HardwareModel(jump_operators=[np.sqrt(gamma) * Observable.sigma_z(0, 1)])

H = Observable.x(0, 1)
sched = Schedule(drive = [(1, H)], tmax=20.)
job = sched.to_job(observable=Observable.z(0, 1))

# instantiating a noisy qpu with trajectories
noisy_qpu_traj = QutipQPU(hardware_model=hw_model, sim_method="stochastic", ntraj=10)
res_noisy_traj = noisy_qpu_traj.submit(job)

# a deterministic run for comparison
noisy_qpu = QutipQPU(hardware_model=hw_model)
res_noisy = noisy_qpu.submit(job)

# the simulator also computes the values of <Z> at intermediate times
for name, result in [("noisy traj", res_noisy_traj), ("noisy", res_noisy)]: 
    print(f"<Z> ({name})= {result.value}")
    times = [float(x) for x in result.value_data.keys()]
    values = [result.value_data[x].re for x in result.value_data.keys()]
    plt.plot(times, values, "-", label=name)
plt.xlabel("$t$")
plt.ylabel(r"$\langle Z\rangle$");
plt.grid()
plt.legend();

<div class="alert alert-block alert-info">
- As you can see, the two curves are not on top. What happens when you increase ``ntraj``?
</div>
